### 通过梯度找到重要的单词 😀

In [3]:

from typing import Sized
import warnings
import os
import torch
import torch.nn as nn
import json
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from transformers import BertConfig, BertTokenizer
from transformers import  BertForMaskedLM
from tqdm import tqdm 
from torch.utils.data import TensorDataset, DataLoader, random_split 
from transformers import BertForSequenceClassification, AdamW 
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, RobertaForSequenceClassification
from transformers import DebertaTokenizer, DebertaModel, DebertaConfig, DebertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup 

import copy
import numpy as np 
from transformers import BertTokenizer, BertModel
import torch
import pickle
import nltk
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [7]:
list1=[[1,2,3],[1,2,3]]
inputs, masks = [
        list(x)
        for x in zip(
            *[i for i in list1]
        )
    ]
print(inputs, masks)

ValueError: too many values to unpack (expected 2)

In [10]:
list1=[[1,2,3],[2,5,7]]
for i, j in zip(*list1):
    print(i,j)

1 2
2 5
3 7


In [4]:
supported_pos_tags = [#nltk词性标注中的一些类别
    #'CC',  # coordinating conjunction, like "and but neither versus whether yet so"  #并列连词
    'JJ',  # Adjective, like "second ill-mannered" #组合式形容词
    'JJR',  # Adjective, comparative, like "colder" #形容词比较级
    'JJS',  # Adjective, superlative, like "cheapest" 形容词最高级
    'NN',  # Noun, singular or mass #名词 单数
    'NNS',  # Noun, plural #名词复数
    'NNP',  # Proper noun, singular #专有名词单数
    'NNPS',  # Proper noun, plural  #专有名词复数
    'RB',  # Adverb#副词
    'RBR',  # Adverb, comparative, like "lower heavier"
    'RBS',  # Adverb, superlative, like "best biggest"
    'VB',  # Verb, base form
    'VBD',  # Verb, past tense 动词过去式
    'VBG',  # Verb, gerund or present participle 动名词/现在分词
    'VBN',  # Verb, past participle #过去分词
    'VBP',  # Verb, non-3rd person singular present #非第三人称单数
    'VBZ',  # Verb, 3rd person singular present #第三人称单数
]

In [5]:
def  get_pos_vocabulary():
    file_name = '/data/zhanghData/AttentionDefense/data/words_pos.dict'
    with open(file_name, 'rb') as f:
            pos_vocabulary = pickle.load(f)
    return pos_vocabulary

In [6]:
pos_vocabulary = get_pos_vocabulary()
print(len(pos_vocabulary))

28301


In [7]:
stop_words = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'almost', 
    'alone', 'along', 'already', 'also', 'although', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anyhow', 
    'anyone', 'anything', 'anyway', 'anywhere', 'are', 'aren', "aren't", 'around', 'as', 'at', 'back', 'been', 'before',
     'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot',
      'could', 'couldn', "couldn't", 'd', 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'down', 'due', 'during', 
      'either', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 
      'first', 'for', 'former', 'formerly', 'from', 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'he', 'hence', 
      'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 
      'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'latter', 
      'latterly', 'least', 'll', 'may', 'me', 'meanwhile', 'mightn', "mightn't", 'mine', 'more', 'moreover', 'most', 'mostly', 
      'must', 'mustn', "mustn't", 'my', 'myself', 'namely', 'needn', "needn't", 'neither', 'never', 'nevertheless', 'next', 
      'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'o', 'of', 'off', 'on', 'once', 'one', 'only', 
      'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'per', 'please', 's', 'same', 
      'shan', "shan't", 'she', "she's", "should've", 'shouldn', "shouldn't", 'somehow', 'something', 'sometime', 'somewhere', 
      'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 
      'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'this', 'those', 'through', 'throughout', 
      'thru', 'thus', 'to', 'too', 'toward', 'towards', 'under', 'unless', 'until', 'up', 'upon', 'used', 've', 'was', 'wasn', 
      "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 
      'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 
      'whom', 'whose', 'why', 'with', 'within', 'without', 'won', "won't", 'would', 'wouldn', "wouldn't", 'y', 'yet', 'you',
       "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', '', ';', 'because', 'tv', "'s", '--', 
       'wo', 'some', '-', 'de', 'ca', 'so', "'ll", "'m", 'despite', 'two', 'should', 'might', "'d", 'inside', 'three', 'be', 'like',
        ')', '.', '...', '``', 'though', 'will', "'", 'each', "''", ',', 'since', 'every', '?', '(', ':', '`', 'us', 'go', '!', 'do',
        'I','So','"','|']

In [9]:
##攻击文件导入
from attack_instance import read_adv_files
atk_path = '/data/zhanghData/AttentionDefense/save_results/attacks/imdb_bert/base/pwws_0.1_100.csv'
atk_instances = read_adv_files(atk_path)
all_orig_texts = [atk.orig_text for atk in atk_instances]
all_perd_texts = [atk.perd_text for atk in atk_instances]
all_gold_labels = [atk.ground for atk in atk_instances]
all_changed_indices = [atk.atk_indices for atk in atk_instances]
all_change_words = [atk.atk_changes for atk in atk_instances] 

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
model.cuda()
load_model_path = '/data/zhanghData/AttentionDefense/save_models/imdb_bert/base/best.pt'
model.load_state_dict(torch.load(load_model_path))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [64]:
model_type = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_type)
config = RobertaConfig.from_pretrained(model_type, num_labels=2, output_attentions=False, output_hidden_states=False, \
                attention_probs_dropout_prob=0, hidden_dropout_prob=0.1)
model = RobertaForSequenceClassification.from_pretrained(model_type, config=config) 
model.cuda()
load_model_path = '/data/zhanghData/AttentionDefense/save_models/sst2_roberta/base/best.pt'
model.load_state_dict(torch.load(load_model_path))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

<All keys matched successfully>

#### 通过获取embedding 重新访问模型 获取embedding.grad

In [7]:
def valid_select(words,sorted_words_idxs,vocabulary,stopwords):
    candidates = [] 
    cand_indices  = []
    for idx in sorted_words_idxs:
        if words[idx] not in vocabulary:
            #print(f"{words[idx]} not in vocabulary")
            vocabulary[words[idx]] = nltk.pos_tag([words[idx]])[0][1]
        if vocabulary[words[idx]] in supported_pos_tags and words[idx] not in stopwords:
            candidates.append(words[idx])
            cand_indices.append(idx)
    return candidates, cand_indices

In [10]:
def create_mask(words,tokenizer,maxlength):
    
    all_input_ids  = []
    for i in range(len(words)):
        text = copy.deepcopy(words)
        text[i]='[MASK]'
        input_ids = tokenizer.encode(
                        " ".join(text),
                        truncation=True,                       
                        add_special_tokens = True,  # 添加special tokens， 也就是CLS和SEP
                        max_length = maxlength,           # 设定最大文本长度 200 for IMDb and 40 for sst
                        # pad_to_max_length = True,   # pad到最大的长度  
                        padding = 'max_length',
                        return_tensors = 'pt'       # 返回的类型为pytorch tensor
                    )
        all_input_ids.append(input_ids)    
    all_input_ids = torch.cat(all_input_ids, dim=0)
    return all_input_ids


In [22]:
def create_mask(words,tokenizer,maxlength):
    
    all_input_ids  = []
    for i in range(len(words)):
        text = copy.deepcopy(words)
        text[i]='[MASK]'
        input_ids = tokenizer.encode(
                        " ".join(text),
                        truncation=True,                       
                        add_special_tokens = True,  # 添加special tokens， 也就是CLS和SEP
                        max_length = maxlength,           # 设定最大文本长度 200 for IMDb and 40 for sst
                        # pad_to_max_length = True,   # pad到最大的长度  
                        padding = 'max_length',
                        return_tensors = 'pt'       # 返回的类型为pytorch tensor
                    )
        all_input_ids.append(input_ids)    
    all_input_ids = torch.cat(all_input_ids, dim=0)
    return all_input_ids
def get_scores(outputs,re_outputs):
    label=torch.argmax(outputs)
    label = int(label)
    confidence = torch.nn.functional.softmax(re_outputs)
    scores = [ float(outputs[label]-logit[label]) for logit in confidence]
    return scores
    #.detach().cpu().numpy()

In [12]:

def get_importance_words(
    model,tokenizer, sentence, pos_vocabulary,stopwords,
    alpha=0.1, max_length=40
    ):
    device = torch.device('cuda')
    model.eval()
    words,sub_words,keys = pre_tokenize(sentence, tokenizer, use_MASK=False, max_length=max_length)
    #print("words:{}".format(words))
    # print("keys:{}".format(keys))
    assert len(words) == len(keys)
    input_words = ['[CLS]'] + sub_words + ['[SEP]']
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(input_words)])
    assert len(sub_words) == (len(input_ids[0])-2)
    outputs = model(input_ids.to(device), token_type_ids=None, attention_mask=(input_ids>0).to(device))
    print(f"predict_class:{torch.argmax(outputs.logits)}")
    mask_copies_ids = create_mask(words,tokenizer,max_length)
    re_outputs = model(input_ids=mask_copies_ids.to(device), token_type_ids=None, \
                    attention_mask=(mask_copies_ids>0).to(device))
    #print(re_outputs)
    scores = get_scores(outputs.logits[0],re_outputs.logits)
    #scores=scores
    
    
    assert len(words) == len(scores)
    #print('------sorting------')
    # for k,value in zip(words,scores):
    #     print(f"({k},{value})")
    sorted_words = sorted(zip(words, scores),key=lambda x:x[1], reverse=True)
    print(f"sorted_words:{sorted_words}")
    sorted_ = sorted(enumerate(scores),key=lambda x:x[1], reverse=True)
    sorted_words_idxs = [i[0] for i in sorted_]
    candidates, cand_idx = valid_select(words,sorted_words_idxs,pos_vocabulary,stopwords)
    #print("After valid:{}".format(candidates))
    result_num = max(min(len(candidates),int(alpha*len(words))),1)
    return candidates[:result_num], cand_idx[:result_num]
    #        sorted_subs = sorted(zip(all_subs_indices,all_sub_scores),key=lambda x:x[1], reverse=True)




In [7]:
def sensitive_test(words,idx,maxlength,tokenizer,model,device):
    input_ids = tokenizer.encode(
                    " ".join(words[:idx]+words[idx+1:]),
                    truncation=True,                       
                    add_special_tokens = True,  # 添加special tokens， 也就是CLS和SEP
                    max_length = maxlength,           # 设定最大文本长度 200 for IMDb and 40 for sst
                    # pad_to_max_length = True,   # pad到最大的长度  
                    padding = 'max_length',
                    return_tensors = 'pt'       # 返回的类型为pytorch tensor
                )
    output = model(input_ids=input_ids.to(device), token_type_ids=None, \
                    attention_mask=(input_ids>0).to(device))
    label= torch.argmax(output.logits[0])
    return int(label)

def create_mask(words,tokenizer,maxlength):
    
    all_input_ids  = []
    for i in range(len(words)):
        text = copy.deepcopy(words)
        text[i]='[MASK]'
        input_ids = tokenizer.encode(
                        " ".join(text),
                        truncation=True,                       
                        add_special_tokens = True,  # 添加special tokens， 也就是CLS和SEP
                        max_length = maxlength,           # 设定最大文本长度 200 for IMDb and 40 for sst
                        # pad_to_max_length = True,   # pad到最大的长度  
                        padding = 'max_length',
                        return_tensors = 'pt'       # 返回的类型为pytorch tensor
                    )
        all_input_ids.append(input_ids)    
    all_input_ids = torch.cat(all_input_ids, dim=0)
    return all_input_ids

def get_scores(outputs,re_outputs):
    scores = []
    label=torch.argmax(outputs)
    label=int(label)
    # print("---------In get scores---------")
    # print(f'label:{label}')
    # print(f"mask labels:{torch.argmax(re_outputs,dim=1)}")
    confidence = torch.nn.functional.softmax(re_outputs)
    #print(confidence)
    for logit in confidence:
        scores.append(float(logit[int(label)]-logit[1-int(label)]))
    # print(scores)
    
    # print("---------endIn get scores---------")
    return scores,label
    #.detach().cpu().numpy()

def get_importance_words(
    model,tokenizer, sentence, label, pos_vocabulary,stopwords,
    alpha=0.1, max_length=40
    ):
    device = torch.device('cuda')
    # model.zero_grad()
    model.eval()
    words,sub_words,keys = pre_tokenize(sentence, tokenizer, use_MASK=False, max_length=max_length)
    #print("words:{}".format(words))
    # print("keys:{}".format(keys))
    assert len(words) == len(keys)
    input_words = ['[CLS]'] + sub_words + ['[SEP]']
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(input_words)])
    assert len(sub_words) == (len(input_ids[0])-2)
    outputs = model(input_ids.to(device), token_type_ids=None, attention_mask=(input_ids>0).to(device))
    #print(f"predict_class:{torch.argmax(outputs.logits)}")
    mask_copies_ids = create_mask(words,tokenizer,max_length)
    re_outputs = model(input_ids=mask_copies_ids.to(device), token_type_ids=None, \
                    attention_mask=(mask_copies_ids>0).to(device))
    #print(re_outputs)
    scores,pre_labbel = get_scores(outputs.logits[0],re_outputs.logits)
    #scores=scores
    attk_flag = 1
    if pre_labbel != label:
        attk_flag = 0
    
    assert len(words) == len(scores)
    #print('------sorting------')
    # for k,value in zip(words,scores):
    #     print(f"({k},{value})")
    sorted_words = sorted(zip(words, scores),key=lambda x:x[1], reverse=True)
    #print(f"sorted_words:{sorted_words}")
    sorted_ = sorted(enumerate(scores),key=lambda x:x[1], reverse=False)
    sorted_words_idxs = [i[0] for i in sorted_]
    candidates, cand_idx = valid_select(words,sorted_words_idxs,pos_vocabulary,stopwords)
    #print("After valid:{}".format(candidates))
    result_num = max(min(len(candidates),int(alpha*len(words))),1)
    
    label_sensitive = sensitive_test(words,cand_idx[0],max_length,tokenizer,model,device)
    sens_flag  = 0
    if (pre_labbel!=label_sensitive):
        sens_flag =1
    return candidates[:2], cand_idx[:2],sens_flag , attk_flag
    #        sorted_subs = sorted(zip(all_subs_indices,all_sub_scores),key=lambda x:x[1], reverse=True)

#### 扰动词的准确率和召回率测试

In [13]:
count_change = 0
count_candidate = 0
count_recall = 0
#count_acc = 0
sens_count = 0
count = 0
suss_count = 0
attk_count = 0
for atk in atk_instances:
    #  self.suss = False
    # self.skip = False
    # self.fail = False
    #print(f"atk.perd_text[:-1]:{atk.perd_text[:-1]}")
    if atk.suss == True:
        count +=1
        print('--------------------------------------------------------------------')
        # print(atk.perd_text)
        # print(f"atk.atk_changes:{atk.atk_changes,atk.atk_indices}")

        # print(f"ture label:{atk.ground},attak label:{atk.perd_label}")
        candidate, cand_idx,sens_flag,attk_flag = get_importance_words\
                (model,tokenizer, atk.perd_text, atk.perd_label,pos_vocabulary,stopwords=stop_words,\
                alpha=0.1, max_length=200)
        
        # print("final important words and idx:{},{}".format(candidate,cand_idx))
        # print(f"true label:{atk.ground}")
        count_change += len(atk.atk_changes)
        count_candidate +=  len(candidate)
        recall = sum([ idx in cand_idx  for idx in atk.atk_indices])
        if recall == 0:
                
                print(atk.perd_text)
                print(f"atk.atk_changes:{atk.atk_changes,atk.atk_indices}")

                print(f"ture label:{atk.ground},attak label:{atk.perd_label}")
                print("final important words and idx:{},{}".format(candidate,cand_idx))
                print(f"true label:{atk.ground}")
        print(f"If sensitive:{sens_flag}")
        print(f"If attack always:{attk_flag}")
        if sens_flag==1 and attk_flag==1:
                suss_count +=1
        count_recall +=recall
        sens_count+=sens_flag
        attk_count += attk_flag
        
acc = count_recall/count_candidate
recall =count_recall/count_change
print("acc:{},recall:{}".format(acc,recall))
print(sens_count)
print(count)

--------------------------------------------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 418.00 MiB (GPU 0; 10.76 GiB total capacity; 6.96 GiB already allocated; 282.44 MiB free; 7.02 GiB reserved in total by PyTorch)

In [88]:
#   suss_count +=1
#         count_recall +=recall
#         sens_count+=sens_flag
#         attk_count += attk_flag
print(attk_count)
print(suss_count)

235
127


In [84]:
count_change = 0
count_candidate = 0
count_recall = 0
#count_acc = 0
sens_count = 0
count = 0
suss_count = 0
attk_count = 0
for atk in atk_instances:
    #  self.suss = False
    # self.skip = False
    # self.fail = False
    #print(f"atk.perd_text[:-1]:{atk.perd_text[:-1]}")
    if atk.suss == True:
        count +=1
        print('--------------------------------------------------------------------')
        # print(atk.perd_text)
        # print(f"atk.atk_changes:{atk.atk_changes,atk.atk_indices}")

        # print(f"ture label:{atk.ground},attak label:{atk.perd_label}")
        candidate, cand_idx,sens_flag,attk_flag = get_importance_words\
                (model,tokenizer, atk.orig_text, atk.ground,pos_vocabulary,stopwords=stop_words,\
                alpha=0.1, max_length=40)
        
        # print("final important words and idx:{},{}".format(candidate,cand_idx))
        # print(f"true label:{atk.ground}")
        count_change += len(atk.atk_changes)
        count_candidate +=  len(candidate)
        recall = sum([ idx in cand_idx  for idx in atk.atk_indices])
        if recall == 0:
                
                print(atk.perd_text)
                print(f"atk.atk_changes:{atk.atk_changes,atk.atk_indices}")

                print(f"ture label:{atk.ground},attak label:{atk.perd_label}")
                print("final important words and idx:{},{}".format(candidate,cand_idx))
                print(f"true label:{atk.ground}")
        print(f"If sensitive:{sens_flag}")
        print(f"If attack always:{attk_flag}")
        if sens_flag==1 and attk_flag==1:
                suss_count +=1
        count_recall +=recall
        sens_count+=sens_flag
        attk_count += attk_flag
        
acc = count_recall/count_candidate
recall =count_recall/count_change
print("acc:{},recall:{}".format(acc,recall))
print("sensity :{}".format(sens_count/count))
print(count)
print(attk_count)
print(suss_count)

--------------------------------------------------------------------
the special effects and many scenes of weightlessness look as good or better than in the original , while the oscar-winning sound and james horner 's rousing score make unspoiled use of the hefty audio system . 
atk.atk_changes:([['good', 'unspoiled']], [29])
ture label:1,attak label:0
final important words and idx:['better', 'original'],[12, 16]
true label:1
If sensitive:0
If attack always:1
--------------------------------------------------------------------
If sensitive:0
If attack always:1
--------------------------------------------------------------------


/home/zhangh/anaconda3/envs/defense/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


If sensitive:1
If attack always:1
--------------------------------------------------------------------
If sensitive:1
If attack always:1
--------------------------------------------------------------------
If sensitive:0
If attack always:1
--------------------------------------------------------------------
this time mr. burns is stressful something in the martin scorsese street-realist mode , but his self-regarding sentimentality trips him up again . 
atk.atk_changes:([['trying', 'stressful']], [5])
ture label:0,attak label:1
final important words and idx:['trips', 'self-regarding'],[18, 16]
true label:0
If sensitive:0
If attack always:1
--------------------------------------------------------------------
you do n't have to know about music to prize the film 's leisurely blend of clowning and mash . 
atk.atk_changes:([['appreciate', 'prize'], ['easygoing', 'leisurely'], ['comedy', 'clowning'], ['romance', 'mash']], [9, 13, 16, 18])
ture label:1,attak label:0
final important words and 

In [ ]:

'''
1.alpha=0.2, max_length=40
score = sum(sub_scores[key[0]:key[1]])/(key[1]-key[0]) 
acc:0.22153529108706851,recall:0.4029990627928772

2.alpha=0.2, max_length=40
score = sum(sub_scores[key[0]:key[1]])
acc:0.29314786192684184,recall:0.5332708528584817

3.alpha=0.1, max_length=40
score = sum(sub_scores[key[0]:key[1]])
acc:0.36455981941309257,recall:0.30271790065604572

3.alpha=0.2, max_length=30
score = sum(sub_scores[key[0]:key[1]])
acc:0.3060869565217391,recall:0.4948453608247423
'''


In [67]:
def get_importance_words_TEMP(
    model,tokenizer, sentence, pos_vocabulary,stopwords,
    alpha=0.1, max_length=40
    ):
    device = torch.device('cuda')
    model.zero_grad()
    words,sub_words,keys = pre_tokenize(sentence, tokenizer, use_MASK=False, max_length=max_length)
    # print("words:{}".format(words))
    # print("keys:{}".format(keys))
    assert len(words) == len(keys)
    input_words = ['[CLS]'] + sub_words + ['[SEP]']
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(input_words)])
    assert len(sub_words) == (len(input_ids[0])-2)
    outputs = model(input_ids.to(device), token_type_ids=None, attention_mask=(input_ids>0).to(device))
    output_label = torch.argmax(outputs.logits)
    
    embeddings_data = getattr(model, 'bert').embeddings.word_embeddings(input_ids.to(device))
    embeddings_data.retain_grad()
    embed_outputs = model(input_ids=None, token_type_ids=None, \
                    attention_mask=(input_ids>0).to(device),inputs_embeds=embeddings_data, labels= output_label)
    #print(f"embed_outputs.keys():{embed_outputs.keys()}") #'loss', 'logits
    embed_outputs.loss.backward()
    #print(f"embedding logit:{embed_outputs.logits}")
    # print(embeddings_data.grad.size())
    scores = torch.mul(embeddings_data,embeddings_data.grad).sum(axis =-1)[0]
    scores = torch.nn.functional.softmax(scores)
    sub_scores  = scores[1:-1].detach().cpu().numpy()
#     sorted_ = sorted(zip(sub_words,scores), key=lambda x:x[1], reverse=True)
    
#     print(sorted_)
    scores = []
    for key in keys:
        
        score = sum(sub_scores[key[0]:key[1]])
        score = sum(sub_scores[key[0]:key[1]])/(key[1]-key[0])
        scores.append(score)
    assert len(words) == len(scores)
    #print('------sorting------')
    # for k,value in zip(words,scores):
    #     print(f"({k},{value})")
    sorted_words = sorted(zip(words, scores),key=lambda x:x[1], reverse=True)
    print(f"sorted_words:{sorted_words}")
    sorted_ = sorted(enumerate(scores),key=lambda x:x[1], reverse=True)
    sorted_words_idxs = [i[0] for i in sorted_]
    candidates, cand_idx = valid_select(words,sorted_words_idxs,pos_vocabulary,stopwords)
    print("After valid:{}".format(candidates))
    result_num = max(min(len(candidates),int(alpha*len(words))),1)
    return candidates[:result_num], cand_idx[:result_num]
    # #        sorted_subs = sorted(zip(all_subs_indices,all_sub_scores),key=lambda x:x[1], reverse=True)



count_change = 0
count_candidate = 0
count_recall = 0
#count_acc = 0
for atk in atk_instances[6:10]:
    #  self.suss = False
    # self.skip = False
    # self.fail = False
    if atk.suss == True:
        print('--------------------------------------------------------------------')
    
        print(f"atk.atk_changes:{atk.atk_changes}")
        #print(f"atk.atk_changes_indices:{atk.atk_indices}")
        candidate, cand_idx = get_importance_words_TEMP\
                (model,tokenizer, atk.perd_text, pos_vocabulary,stopwords=stop_words,\
                alpha=0.2, max_length=30)
        
        print("final important words and idx:{},{}".format(candidate,cand_idx))

        count_change += len(atk.atk_changes)
        count_candidate +=  len(candidate)
        count_recall +=sum([ idx in cand_idx  for idx in atk.atk_indices ])
        
acc = count_recall/count_candidate
recall =count_recall/count_change
print("acc:{},recall:{}".format(acc,recall))

--------------------------------------------------------------------
atk.atk_changes:[['dazzles', 'stuns'], ['best', 'upper'], ['years', 'aging']]
sorted_words:[('stuns', 0.04027263633906841), ('johnny', 0.03610336408019066), ('always', 0.036089032888412476), ('relates', 0.03604688495397568), ('and', 0.03592899814248085), (',', 0.03585952892899513), ('(', 0.03579616919159889), ('to', 0.03575718402862549), ('fully-written', 0.03574215744932493), ('which', 0.0356290265917778), ('characters', 0.03562147542834282), ('its', 0.03547978773713112), ('and', 0.03535338118672371), ('its', 0.03512479364871979), ('determined', 0.03505651280283928), ('stylishness', 0.03482627682387829), ('with', 0.03481762111186981), (')', 0.034306179732084274), ('characters', 0.0341782346367836), ('story', 0.03391190245747566)]
After valid:['stuns', 'johnny', 'always', 'relates', 'fully-written', 'characters', 'determined', 'stylishness', 'characters', 'story']
final important words and idx:['stuns', 'johnny', 'alw

/home/zhangh/anaconda3/envs/defense/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [113]:
model.zero_grad()
outputs = model(input_ids.to(device), token_type_ids=None, attention_mask=(input_ids>0).to(device))
print(outputs.keys())
logit = outputs.logits
print(f"logit:{logit}")
output_label = torch.argmax(logit)
print(output_label)

embeddings_data = getattr(model, 'bert').embeddings.word_embeddings(input_ids.to(device))
embeddings_data.retain_grad()
embed_outputs = model(input_ids=None, token_type_ids=None, \
                attention_mask=(input_ids>0).to(device),inputs_embeds=embeddings_data, labels= output_label)
print(f"embed_outputs.keys():{embed_outputs.keys()}")
embed_outputs.loss.backward()
print(f"embedding logit:{embed_outputs.logits}")
# print(embeddings_data.grad.size())
scores = torch.mul(embeddings_data,embeddings_data.grad).sum(axis =-1)[0]
scores = torch.nn.functional.softmax(scores)
scores  = scores.detach().cpu().numpy()
sorted_ = sorted(zip(input_words,scores), key=lambda x:x[1], reverse=True)
print(sorted_)



odict_keys(['logits'])
logit:tensor([[ 2.4144, -2.0190]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor(0, device='cuda:0')
embed_outputs.keys():odict_keys(['loss', 'logits'])
embedding logit:tensor([[ 2.0845, -1.6242]], device='cuda:0', grad_fn=<AddmmBackward>)
[('film', 0.114470914), ('elliptical', 0.11247816), (',', 0.11201403), ('gross', 0.11179187), ('[CLS]', 0.111171745), (',', 0.11104514), ('[SEP]', 0.10929688), ('a', 0.10909306), ('quiet', 0.10863828)]


/home/zhangh/anaconda3/envs/defense/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#### 类似于pgd对抗训练的方法： 获取embedding.grad

In [25]:
from refactor import pre_tokenize
device = torch.device('cuda')
sentence = 'a quiet , nice , elliptical film '
sentence = 'a quiet , pure , elliptical film '
sentence = "a quiet , plain , elliptical film"
sentence = 'the second coming of harry potter is a film far worse to its predecessor'
words,sub_words,keys = pre_tokenize(sentence, tokenizer, use_MASK=False, max_length=40)
assert len(words) == len(keys)
input_words = ['[CLS]'] + sub_words + ['[SEP]']
print(len(input_words))
print(input_words)
input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(input_words)])
print(input_ids)


16
['[CLS]', 'the', 'second', 'coming', 'of', 'harry', 'potter', 'is', 'a', 'film', 'far', 'worse', 'to', 'its', 'predecessor', '[SEP]']
tensor([[  101,  1996,  2117,  2746,  1997,  4302, 10693,  2003,  1037,  2143,
          2521,  4788,  2000,  2049,  8646,   102]])


In [31]:
words,sub_words,keys = pre_tokenize(sentence, tokenizer, use_MASK=False, max_length=40)

In [34]:
keys

[[0, 1],
 [1, 2],
 [2, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 12],
 [12, 13],
 [13, 14]]

In [26]:
#model.zero_grad()
outputs = model(input_ids.to(device), token_type_ids=None, attention_mask=(input_ids>0).to(device))
print(outputs.keys())
logit = outputs.logits
print(f"logit:{logit}")
output_label = torch.argmax(logit)
print(f"output_label:{output_label}")

re_outputs =model(input_ids.to(device), token_type_ids=None, attention_mask=(input_ids>0).to(device), labels= output_label)
print(f"re_outputs.keys():{re_outputs.keys()}")
re_outputs.loss.backward()
print(f"re logit:{re_outputs.logits}")

odict_keys(['logits'])
logit:tensor([[ 3.2983, -3.1494]], device='cuda:0', grad_fn=<AddmmBackward0>)
output_label:0
re_outputs.keys():odict_keys(['loss', 'logits'])
re logit:tensor([[ 3.2983, -3.1494]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [27]:
emb_name = 'word_embeddings'
for name, param in model.named_parameters():
    #print(name)
    if param.requires_grad and emb_name in name: 
        print(param.is_leaf) 
        
        word_embed=param.data
        word_embed_grad = param.grad
        print(param.grad.sum())
        break

True
tensor(1.8440e-07, device='cuda:0')


In [28]:

input_embed = torch.index_select(word_embed,0,input_ids[0].to(device)).to(device)
input_embed_grad = torch.index_select(word_embed_grad,0,input_ids[0].to(device)).to(device)


In [29]:
input_embed_grad.size()

torch.Size([16, 768])

In [30]:
# print(input_embed)
# print(input_embed_grad)
#scores = torch.mul(input_embed,input_embed_grad).sum(axis =-1)
scores = (input_embed_grad).sum(axis =-1)
print(scores)
# scores = torch.nn.functional.softmax(scores)
# print(scores )
scores  = scores.detach().cpu().numpy()
sorted_ = sorted(zip(input_words,scores), key=lambda x:x[1], reverse=True)
print(sorted_)

tensor([ 5.9605e-08,  1.4901e-08,  2.2352e-08, -2.9802e-08, -1.8626e-09,
        -2.2352e-08, -5.9605e-08,  1.4901e-08,  1.4901e-08, -2.2352e-08,
         1.4901e-08,  4.6566e-10, -1.4901e-08, -2.7940e-08,  3.3528e-08,
         3.7253e-09], device='cuda:0')
[('[CLS]', 5.9604645e-08), ('predecessor', 3.3527613e-08), ('second', 2.2351742e-08), ('the', 1.4901161e-08), ('is', 1.4901161e-08), ('a', 1.4901161e-08), ('far', 1.4901161e-08), ('[SEP]', 3.7252903e-09), ('worse', 4.656613e-10), ('of', -1.8626451e-09), ('to', -1.4901161e-08), ('harry', -2.2351742e-08), ('film', -2.2351742e-08), ('its', -2.7939677e-08), ('coming', -2.9802322e-08), ('potter', -5.9604645e-08)]


In [ ]:
pre_tokenize(sentence, tokenizer, use_MASK=False)

In [40]:
scores*10e7
scores = torch.nn.functional.softmax(torch.tensor(scores*10e7))
print(scores)

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


/anaconda3/envs/defense/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/anaconda3/envs/defense/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [87]:
scores_ = torch.nn.functional.softmax(scores)
print(scores_)
#['[CLS]', 'a', 'quiet', ',', 'gross', ',', 'elliptical', 'film', '[SEP]']

tensor([0.0571, 0.1451, 0.0798, 0.0882, 0.0137, 0.0605, 0.4436, 0.0208, 0.0912],
       device='cuda:0', grad_fn=<SoftmaxBackward>)


/home/zhangh/anaconda3/envs/defense/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


### create stop_words set  ☀fighting!

In [44]:
def get_sst_data(type='train'):
    '''SST-2 GLUE version
    '''
    print('Getting Data...')
    texts = [] 
    labels = [] 
    #path = './data/sst2-mini/' + type + '.tsv'
    path = './data/sst2/' + type + '.tsv'
    with open(path, 'r', encoding='utf8') as fin:
        for line in fin.readlines()[1:]:
            line = line.strip().split('\t')
            texts.append(line[0])
            labels.append(1. if line[1]=='1' else 0.)
    print('Done, load {} datas from sst2 train dataset.'.format(len(texts)))
    return texts, labels 

In [55]:
test_texts,test_labels = get_sst_data(type='dev')
train_texts,train_labels = get_sst_data(type='train')

Getting Data...
Done, load 872 datas from sst2 train dataset.
Getting Data...
Done, load 67349 datas from sst2 train dataset.


In [57]:
vocabulary = {}
for text in test_texts+train_texts:
    words = text.split(" ")
    for w in words:
        if w not in vocabulary:
            vocabulary[w] = 1
        else:
            vocabulary[w] += 1

['/data/zhanghData/AttentionDefense/s',
 'ave_results/dynamic_defense/sst2_bert/base/textfooler_0.2_100_topk5_alpha0.2.txt']

In [80]:
import nltk
supported_pos_tags = [#nltk词性标注中的一些类别
    #'CC',  # coordinating conjunction, like "and but neither versus whether yet so"  #并列连词
    'JJ',  # Adjective, like "second ill-mannered" #组合式形容词
    'JJR',  # Adjective, comparative, like "colder" #形容词比较级
    'JJS',  # Adjective, superlative, like "cheapest" 形容词最高级
    'NN',  # Noun, singular or mass #名词 单数
    'NNS',  # Noun, plural #名词复数
    'NNP',  # Proper noun, singular #专有名词单数
    'NNPS',  # Proper noun, plural  #专有名词复数
    'RB',  # Adverb#副词
    'RBR',  # Adverb, comparative, like "lower heavier"
    'RBS',  # Adverb, superlative, like "best biggest"
    'VB',  # Verb, base form
    'VBD',  # Verb, past tense 动词过去式
    'VBG',  # Verb, gerund or present participle 动名词/现在分词
    'VBN',  # Verb, past participle #过去分词
    'VBP',  # Verb, non-3rd person singular present #非第三人称单数
    'VBZ',  # Verb, 3rd person singular present #第三人称单数
]
temp = []
for key,value in sorted_:
    if len(key) <= 2:
        temp.append(key)
    elif nltk.pos_tag([key])[0][1] not in supported_pos_tags:
        temp.append(key)
    if value <100:
        break

In [81]:
temp = list(set(temp))
print(temp)

['', 'you', 'my', ';', 'because', 'they', 'would', 'its', 'under', 'tv', 'off', 'themselves', 'if', 'must', 'our', "'s", '--', 'neither', 'is', 'a', 'wo', 'against', 'some', 'me', 'may', 'them', '-', 'de', 'all', 'one', 'could', 'ca', 'so', 'without', "'ll", 'either', "'m", 'despite', 'and', 'from', 'two', 'should', 'above', 'along', 'might', 'on', "'d", 'around', 'over', 'what', 'him', 'inside', 'three', 'i', 'in', 'your', 'be', 'up', 'nor', 'her', 'like', ')', 'while', 'between', 'for', 'how', 'where', 'before', 'through', 'by', 'out', 'to', 'these', 'another', '.', 'an', '...', 'who', '``', 'though', 'itself', 'whose', 'will', 'as', 'than', 'he', "'", 'why', 'any', 'each', 'both', 'this', "''", 'no', ',', 'but', 'the', 'it', 'we', 'since', 'after', 'that', 'every', '?', 'his', 'or', 'she', 'at', '(', 'with', 'himself', 'those', 'their', 'about', ':', 'beyond', '`', 'into', 'of', 'us', 'when', 'go', '!', 'which', 'do', 'can']


In [60]:
from utils.refactor import get_stopwords

In [62]:
stopwords= get_stopwords()

In [82]:
for t in temp:
    if t not in stopwords:
        stopwords.append(t)


In [2]:
from utils.refactor import get_stopwords
stop_words = get_stopwords()
print(f"stopwords:{len(stop_words)}")


stopwords:1


In [3]:
print(stop_words)

(['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'aren', "aren't", 'around', 'as', 'at', 'back', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot', 'could', 'couldn', "couldn't", 'd', 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'down', 'due', 'during', 'either', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 'first', 'for', 'former', 'formerly', 'from', 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'isn', "isn't", 'it',